In [68]:
test_input = """
...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....
"""
with open('day11.txt', 'rt') as f: raw_input = f.read()

In [88]:
from collections import namedtuple
from math import sqrt
from queue import PriorityQueue
from itertools import combinations

Action = namedtuple('Action', ['dx', 'dy', 'name'])
Position = namedtuple('Location', ['x', 'y'])

ACTIONS = [
    Action(-1, 0, 'left'),
    Action(1, 0, 'right'),
    Action(0, 1, 'up'),
    Action(0, -1, 'down'),
]

Matrix = list[str]

STAR = '#'
SPACE = '.'
CURRENT = '$'
TARGET = '*'

class State:
    def __init__(self, h: int, pos: Position, steps: list[Position] = None):
        self.steps = steps or []
        self.h = h
        self.pos = pos
    
    @property
    def g(self):
        return len(self.steps)
    
    @property
    def f(self):
        return self.g + self.h

    def __gt__(self, other):
        return self.f > other.f

def is_valid(matrix: Matrix, pos: Position) -> bool:
    assert len(matrix) != 0, 'Empty matrix'
    if pos.x < 0 or pos.y < 0 or pos.x > (len(matrix[0]) - 1) or pos.y > (len(matrix) - 1):
        return False
    return True

def get_item(matrix: Matrix, pos: Position) -> str:
    if not is_valid(matrix, pos):
        raise KeyError('Invalid position')
    return matrix[pos.y][pos.x]

def is_moveable(matrix: Matrix, pos: Position, end: Position = None) -> bool:
    try:
        item = get_item(matrix, pos)
    except KeyError:
        return False
    return item != STAR or pos == end

def heuristic(start: Position, end: Position) -> float:
    return abs(end.x - start.x)+ abs(end.y - start.y)

def astar_search(matrix: Matrix, start: Position, end: Position) -> list[Position]:
    stacks: PriorityQueue[State] = PriorityQueue()
    stacks.put(State(h=0, pos=start))
    closed: set[Position] = set()
    
    steps = None
    while stacks:
        current_state = stacks.get()
        current_pos = current_state.pos
        if current_pos in closed:
            continue
        elif current_pos == end:
            steps = current_state.steps
            break
        else:
            for action in ACTIONS:
                next_pos = Position(current_pos.x + action.dx, current_pos.y + action.dy)
                if not is_moveable(matrix, next_pos, end):
                    continue
                else:
                    next_state = State(
                        h=heuristic(next_pos, end),
                        pos=next_pos,
                        steps=[*current_state.steps, current_pos]
                    )
                    stacks.put(next_state)
    return steps

def parse(raw_input: str) -> Matrix:
    matrix = list(map(list, raw_input.strip().split()))
    expand_rows = []
    expand_columns = []
    for column_index in range(len(matrix[0])):
        is_expand = True
        for row in matrix:
            if row[column_index] != SPACE:
                is_expand = False
                break
        if is_expand:
            expand_columns.append(column_index)
    for row_index in range(len(matrix)):
        row = matrix[row_index]
        is_expand = True
        for item in row:
            if item != SPACE:
                is_expand = False
                break
        if is_expand:
            expand_rows.append(row_index)
    for count, row_index in enumerate(expand_rows):
        current_row_index = count + row_index
        expand_row = matrix[current_row_index][::]
        matrix.insert(current_row_index, expand_row)
    
    for count, column_index in enumerate(expand_columns):
        current_column_index = count + column_index
        for row in matrix:
            row.insert(current_column_index, SPACE)
    matrix = [''.join(row) for row in matrix]
    return matrix

def find_stars(matrix: Matrix) -> list[Position]:
    stars = []
    for row_index, row in enumerate(matrix):
        for column_index, item in enumerate(row):
            if item == STAR:
                stars.append(Position(x=column_index, y=row_index))
    return stars

def solve1(raw_input: str) -> int:
    matrix = parse(raw_input)
    stars = find_stars(matrix)
    return sum([heuristic(*pair) for pair in combinations(stars, 2)])

def parse2(raw_input: str) -> Matrix:
    matrix = raw_input.strip().split()
    expand_rows = []
    expand_columns = []
    for column_index in range(len(matrix[0])):
        is_expand = True
        for row in matrix:
            if row[column_index] != SPACE:
                is_expand = False
                break
        if is_expand:
            expand_columns.append(column_index)
    for row_index in range(len(matrix)):
        row = matrix[row_index]
        is_expand = True
        for item in row:
            if item != SPACE:
                is_expand = False
                break
        if is_expand:
            expand_rows.append(row_index)
    return matrix, expand_rows, expand_columns

def distance(
    start: Position, end: Position, expand_rows: list[int], expand_columns: list[int], expand_ratio: int = 1
):
    dexpand = expand_ratio - 1
    dx = abs(end.x - start.x)
    for x in expand_columns:
        if min(start.x, end.x) < x < max(start.x, end.x):
            dx += dexpand
    dy = abs(end.y - start.y)
    for y in expand_rows:
        if min(start.y, end.y) < y < max(start.y, end.y):
            dy += dexpand
    return dx + dy

def solve2(raw_input: str) -> int:
    matrix, expand_rows, expand_columns = parse2(raw_input)
    stars = find_stars(matrix)
    return sum([
        distance(*pair, expand_rows=expand_rows, expand_columns=expand_columns, expand_ratio=1000000) 
        for pair in combinations(stars, 2)
    ])
        

In [89]:
solve2(raw_input)

569052586852